## Importar dataframe base

In [1]:
import pandas as pd
import numpy as np

# skempi db, a partir da qual será gerada a nossa
skempi = pd.read_csv("skempi_v2.csv", delimiter=";")
# df, nossa df já filtrada
df = pd.DataFrame()

## Limpar dados

### Apenas entradas com mutação na alanina

In [3]:
skempi = skempi[skempi['Mutation(s)_PDB'].str.endswith('A')].reset_index(drop=True)

### Remover indicação de afinidade maior ou menor que X 
Algumas entradas tem o seguinte formato:
Affinity_mut (M) = '<5466'

In [4]:
# TODO não sei se devo fazer isso ou remover os dados
replace_gt_lt = lambda data: data.str.replace('[<>]', '', regex=True)
skempi[['Affinity_mut (M)', 'Affinity_wt (M)']] = skempi[['Affinity_mut (M)', 'Affinity_wt (M)']].apply(replace_gt_lt)
                                                                                                                
## Solução removendo-os
# skempi_ala['Affinity_mut (M)'] = skempi_ala.loc[~skempi_ala['Affinity_mut (M)'].str.startswith(('<', '>')), 'Affinity_mut (M)']
# skempi_ala['Affinity_wt (M)'] = skempi_ala.loc[~skempi_ala['Affinity_wt (M)'].str.startswith(('<', '>')), 'Affinity_wt (M)']

### Converter valores de afinidade para float

Os valores de afinidade estão sendo interpretados como object. É preciso converter para float

In [5]:
skempi[['Affinity_mut (M)', 'Affinity_wt (M)']] = skempi[['Affinity_mut (M)', 'Affinity_wt (M)']].apply(pd.to_numeric, errors='coerce')

Algumas entradas não tem dados em 'Affinity_mut (M)', tornando-as inuteis para verificação de hotspots. É preciso remove-las

In [13]:
skempi = skempi.dropna(subset=['Affinity_mut (M)'])

## Selecinar campos desejados

### pdb_id, partner1, partner2

In [ ]:
df[['pdb_id', 'partner1', 'partner2']] = skempi['#Pdb'].str.extract('(\w+)_(\w+)_(\w+)')

### partner_mut, resnum

In [ ]:
df['partner_mut']= skempi['Mutation(s)_PDB'].str[1]

In [ ]:
df['resnum'] = skempi['Mutation(s)_PDB'].str.extract('(\d+)')

certas entradas apresentam mutação em ambos participantes

### ΔG_wt, ΔG_mut, ΔΔG,  hotspot

Segundo o site da [documentação](https://life.bsc.es/pid/skempi2/info/faq_and_help#5) do banco de dados
>The affinities (Kd) of the wild-type complexes are in the column 'affinity_wt' and the affinities of the mutant are in the column 'affinity_mut'. These can be converted to ΔG values by the relationship **ΔG = RT ln Kd**; at room temperature this is **ΔG = (8.314/4184)*(273.15 + 25.0) * ln(wt)**, where ln() is the natural logarithm. The changes in affinity upon mutation is calculated as **ΔΔG = ΔGmut-ΔGwt**.
>

In [ ]:
# Definir funções ΔG e ΔΔG
# Talvez seja necessário alterar valores de temperatura

calculate_dG = lambda kd: ((8.314/4184)*(273.15 + 25.0) * np.log(kd))
calculate_ddG =  lambda dG_mut, dG_wt: dG_mut - dG_wt

In [ ]:
df['dG_mut'] = calculate_dG(skempi_ala['Affinity_mut (M)'])
df['dG_wt'] = calculate_dG(skempi_ala['Affinity_wt (M)'])

In [ ]:
df['ddG'] = calculate_ddG(df['dG_mut'], df['dG_wt'])

In [ ]:
df['hotspot'] = (df['ddG'] > 2.5).astype(int)

## Salvar nova tabela em csv

In [ ]:
df.to_csv('alalin_filtered_skempi.csv', sep=';', index=False)